Prvním krokem k tvorbě naší aplikace bude vytvoření virtuálního prostředí jazyka Python3. Virtuální prostředí představuje oddělenou "větev" nainstalovaného interpreteru a umožňuje instalovat odlišné moduly tak, že k nim hlavní instance Pythonu nemá přístup. Virutální prostředí můžeme vytvořit pomocí následujícího příkazu v terminálu či příkazovém řádku:

`$ python3 -m venv .venv`

Následně je pro použití nutné prostředí aktivovat (způsob se liší podle platformy)

macOS/Linux (bash/zsh): <br>
`$ source .venv/bin/activate`

Windows (cmd): <br>
`C:\> <venv>\Scripts\activate.bat`

Windows (PowerShell): <br>
`C:\> <venv>\Scripts\activate.ps1`

Aktivované prostředí indikuje jeho název v závorce na začátku příkazového řádku: <br>
`(.venv) myaccount@mymachine ProjectDirectory %`

Moduly instalujeme běžně příkazem `pip install <mymodule>` a programy spouštíme `python <myapp>`

Do virtuálního prostředí budeme instalovat veškeré potřebné moduly a budeme ho používat ke spuštění naší aplikace. Prvním krokem k tvorbě aplikace bude zdroj dat, která budeme následně pomocí webového prostředí uživateli prezentovat.
K získávání dat o poloze a počasí budeme využívat API odpovídající standardu REST od poskytovatele OpenWeatherMap. Standard REST(RESTful) specifikuje provozovateli a tvůrci API pravidla, podle kterých má být API vystavěna a provozována. Pro nás jakožto koncového uživatele, který implementuje API ve své aplikaci, je však relevantní pouze informace, že "reprezentace" zdroje, tedy informace dodané API, jsou ve standardním formátu JSON. Pro používání API je také důležité znát pojem parametrů, které jsou unikátní pro jednotlivé typy HTTP dotazů na API. Konkrétní parametry specifikuje provozovatel API, v našem případě OpenWeatherMap.

Více o standardu REST: <https://www.redhat.com/en/topics/api/what-is-a-rest-api>

API OpenWeatherMap: <https://openweathermap.org/api> 

Prvním krokem k využití OWM API je registrace, po které Vám bude přidělen unikátní identifikátor, tzv. API Key. Jedná se o klíčový parametr pro všechny dotazy na API a je unikátní pro každého registrovaného uživatele. 

Po registraci a získání API klíče je nutné zjistit, které konkrétní parametry budeme muset API dodat tak, abychom zíkali potřebné informace o poloze a počasí.

Pro získávání meteorologických dat budeme využívat Weather API, pro zeměpisná data pak Geocoding API.

In [2]:
weather_api_url = "https://api.openweathermap.org/data/2.5/weather"
geocoding_api_url = "http://api.openweathermap.org/geo/1.0/direct"

Za účelem volání HTTP dotazů existuje pro jazyk Python modul `requests` <https://pypi.org/project/requests/>, kterou si nainstalujeme a importujeme do naší aplikace

`pip install requests`<br>
`import requests`

Další důležitou částí jsou data ve formátu JSON. Formát JSON (JavaScript Object Notation) je hojně využívaný z důvodu své jednoduché čitelnosti jak programátory, tak počítači. I přes svůj název se jedná o jazykově nezávislou strukturu, jejíž datové podstruktuy odpovídají běžným datovým strukturám využívaným ve většině moderních programovacích jazyků. V jazyce Python se jedná o slovníky (dictionary) a seznamy (list). <br>

<https://www.json.org/json-en.html><br>

Pro práci s formátem JSON nabízí modul requests vestavěnou metodu `.json()`, alternativně můžeme však využít celý vestavěný modul `json`, který nabízí širší funkcionalitu.

Posledním krokem před prvním voláním dotazu API je tvorba souboru s proměnnou prostředí, která bude uchovávat náš unikátní API klíč. Ten bychom neměli v žádném případě odhalovat ve zdrojovém kódu naší aplikace. Proměnná prostředí je souborem `.env`, který obsahuje data v řádcích v následujícím formátů: "MY_API_KEY=<api_key>". 

Vytvořit ho můžeme pomocí terminálu či přímo modulu jazyka Python `os`

macOS/Linux:

`$ touch .env` <br>
`$ echo "USER_KEY=<api_key>" >> .env`

In [ ]:
api_key = "můj API klíč"
import os
os.system("touch .env")
os.system(f'echo "USER_KEY={api_key}" >> .env')

V aplikaci pak jednotlivé proměnné prostředí přecteme pomocí modulu python-decouple <https://pypi.org/project/python-decouple/>, který slouží k oddělení nastavení aplikace od samotného zdrojového kódu.

`pip install python-decouple` <br>
`from decouple import config`

Tím můžeme pokročit k poslání prvního dotazu na OpenWeatherMap API.

In [3]:
import requests
import json
from decouple import config

KEY = config('USER_KEY')
weather_api_url = "https://api.openweathermap.org/data/2.5/weather"

# jako souřadnice budeme používat přibližné souřadnice středu Čech
lat = 50
lon = 14

# veškeré parametry HTTP dotazu umístíme do dictionary kde key parametru bude jeho název specifikovaný provozovatelem API

params = {
    "APPID":KEY,
    "lat":lat,
    "lon":lon
}

# nyní můžeme poslat samotnou žádost

response = requests.get(weather_api_url, params=params)

In [ ]:
# pomocí statusových kódu HTTP žádosti můžeme zjistit, zda proběhlo volání API bez problémů, kód 200 signalizuje správný průběh

response.status_code

200

In [5]:
# nyní si zobrazíme obsah vrácený HTTP dotazem

response.content

b'{"coord":{"lon":14,"lat":50},"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04d"}],"base":"stations","main":{"temp":301.82,"feels_like":300.76,"temp_min":300.05,"temp_max":304.24,"pressure":1016,"humidity":30,"sea_level":1016,"grnd_level":989},"visibility":10000,"wind":{"speed":1,"deg":58,"gust":2.3},"clouds":{"all":67},"dt":1658678737,"sys":{"type":1,"id":6848,"country":"CZ","sunrise":1658632924,"sunset":1658689116},"timezone":7200,"id":3069738,"name":"Ni\xc5\xbebor","cod":200}'

In [6]:
# obsah převedeme na python objekt JSON, který se chová jako běžný dictionary

data = json.loads(response.content)
data

{'coord': {'lon': 14, 'lat': 50},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 301.82,
  'feels_like': 300.76,
  'temp_min': 300.05,
  'temp_max': 304.24,
  'pressure': 1016,
  'humidity': 30,
  'sea_level': 1016,
  'grnd_level': 989},
 'visibility': 10000,
 'wind': {'speed': 1, 'deg': 58, 'gust': 2.3},
 'clouds': {'all': 67},
 'dt': 1658678737,
 'sys': {'type': 1,
  'id': 6848,
  'country': 'CZ',
  'sunrise': 1658632924,
  'sunset': 1658689116},
 'timezone': 7200,
 'id': 3069738,
 'name': 'Nižbor',
 'cod': 200}

In [7]:
# nyní můžeme z JSON objektu získávat jednotlivé údaje o počasí v naší lokalitě

data["weather"][0]["description"]

'broken clouds'

<h1>Cvičení 1</h1>

Vytvořte program, který umožní uživateli zadat souřadnice a na jejich základě získá informace o počasí v jejich lokalitě. Ty úhledně vypíše na výstupu. Bude se jednat o název místa, zemi, slovní popis počasí, teplotu (v °C) a rychlost větru (v km/h). Pro detaily k jednotlivým údajům konzultujte oficiální dokumentaci OpenWeatherMap.

In [ ]:
# místo pro váš kód

<h1>Vzorové řešení 1</h1>

In [3]:
# spusťte pro kontrolu funkcionality

import json
import requests
from decouple import config

KEY = config('USER_KEY')

def get_user_input():
    values = []
    for c in ["latitude", "longtitude"]:
        try:
            val = float(input(f"Please enter the {c} of your location: "))
        except ValueError:
            print("Error - not a number, please retry")
            val = float(input(f"Please enter the {c} of your location: "))
        values.append(val)         
    return values

def parse_data(lat_lon):
    api_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"APPID": KEY}
    params["lat"], params["lon"] = lat_lon
    response = requests.get(api_url,params=params) 
    return json.loads(response.content) if response.status_code == 200 else print("Uknown error when sending HTTP request")
        
def extract_weather(data):
    city = data["name"]
    country = data["sys"]["country"]
    desc = data["weather"][0]["description"]
    temp_raw = data["main"]["temp"] - 273.15
    wind_raw = data["wind"]["speed"] * 3.6
    temp = str(round(temp_raw, 2)) + " °C"
    wind = str(round(wind_raw,2)) + " km/h"
    return [city, country, desc, temp, wind]

def print_all(_list):
    for x in _list:
        print(x)
        
def main():
    lat_lon = get_user_input()
    weather_data = parse_data(lat_lon)
    to_print = extract_weather(weather_data)
    print_all(to_print)

if __name__ == '__main__':
    main() 

Vrsar
HR
clear sky
27.22 °C
11.84 km/h


Nyní jsme se naučili používat API na získávání informací o počasí, v aplikaci však nebudeme využívat pouze tuto API. Další potřebnou součástí je tzv. Geocoding API. Ta nám umožňuje získavat zeměpisné informace na základě vstupních dat determinujících konkrétní místo. Zpravidla se jedná buď o souřadnice daného bodu, či název města, státu (v USA) a země. 

<https://openweathermap.org/api/geocoding-api>

Pro získávání souřadnic daného bodu z názvu místa budeme využívat funkci "direct geocoding", která nám vrátí soubor JSON obsahující souřadnice místa. Tato funkcionalita nám později v aplikaci umožní vyhledávání počasí podle názvu místa, což může být pro některé uživatele přívětivější, než hledání na mapě. Na základě dokumentace k této funkci můžeme zjistit, že potřebné parametry jsou ve formátu `q={název města}, {kód státu (pro USA)}, {kód země}`

In [6]:
import requests
import json
from decouple import config

KEY = config('USER_KEY')
direct_geocoding_api_url = "http://api.openweathermap.org/geo/1.0/direct"

# jako název budeme používat Prahu a Českou republiku

query = "Praha, CZ" 

# veškeré parametry HTTP dotazu umístíme do dictionary kde key parametru bude jeho název specifikovaný provozovatelem API

params = {
    "APPID": KEY,
    "q":query
}

# nyní můžeme poslat samotnou žádost

response = requests.get(direct_geocoding_api_url, params=params)


In [7]:
# pomocí statusových kódu HTTP žádosti můžeme zjistit, zda proběhlo volání API bez problémů, kód 200 signalizuje správný průběh

response.status_code

200

In [8]:
# nyní si zobrazíme obsah vrácený HTTP dotazem

response.content

b'[{"name":"Prague","local_names":{"ro":"Praga","mk":"\xd0\x9f\xd1\x80\xd0\xb0\xd0\xb3\xd0\xb0","jv":"Praha","hy":"\xd5\x8a\xd6\x80\xd5\xa1\xd5\xb0\xd5\xa1","mn":"\xd0\x9f\xd1\x80\xd0\xb0\xd0\xb3\xd0\xb0","vi":"Praha","sq":"Praga","mr":"\xe0\xa4\xaa\xe0\xa5\x8d\xe0\xa4\xb0\xe0\xa4\xbe\xe0\xa4\x97","az":"Praqa","tr":"Prag","en":"Prague","fr":"Prague","ca":"Praga","et":"Praha","zh":"\xe5\xb8\x83\xe6\x8b\x89\xe6\xa0\xbc","io":"Praha","oc":"Praga","uk":"\xd0\x9f\xd1\x80\xd0\xb0\xd0\xb3\xd0\xb0","tt":"\xd0\x9f\xd1\x80\xd0\xb0\xd0\xb3\xd0\xb0","se":"Praha","sk":"Praha","mi":"Prague","am":"\xe1\x8d\x95\xe1\x88\xab\xe1\x8c\x8d","af":"Praag","cy":"Prag","bn":"\xe0\xa6\xaa\xe0\xa7\x8d\xe0\xa6\xb0\xe0\xa6\xbe\xe0\xa6\x97","gv":"Praag","pt":"Praga","fy":"Praach","co":"Praga","eo":"\xc4\x88efurba regiono Prago","ak":"Prague","gl":"Praga","nn":"Praha","zu":"IPraha","ka":"\xe1\x83\x9e\xe1\x83\xa0\xe1\x83\x90\xe1\x83\xa6\xe1\x83\x90","kw":"Praha","yo":"Prague","ja":"\xe3\x83\x97\xe3\x83\xa9\xe3\x83\x8

In [9]:
# obsah převedeme na python objekt JSON, který se chová jako běžný dictionary

data = json.loads(response.content)
data

[{'name': 'Prague',
  'local_names': {'ro': 'Praga',
   'mk': 'Прага',
   'jv': 'Praha',
   'hy': 'Պրահա',
   'mn': 'Прага',
   'vi': 'Praha',
   'sq': 'Praga',
   'mr': 'प्राग',
   'az': 'Praqa',
   'tr': 'Prag',
   'en': 'Prague',
   'fr': 'Prague',
   'ca': 'Praga',
   'et': 'Praha',
   'zh': '布拉格',
   'io': 'Praha',
   'oc': 'Praga',
   'uk': 'Прага',
   'tt': 'Прага',
   'se': 'Praha',
   'sk': 'Praha',
   'mi': 'Prague',
   'am': 'ፕራግ',
   'af': 'Praag',
   'cy': 'Prag',
   'bn': 'প্রাগ',
   'gv': 'Praag',
   'pt': 'Praga',
   'fy': 'Praach',
   'co': 'Praga',
   'eo': 'Ĉefurba regiono Prago',
   'ak': 'Prague',
   'gl': 'Praga',
   'nn': 'Praha',
   'zu': 'IPraha',
   'ka': 'პრაღა',
   'kw': 'Praha',
   'yo': 'Prague',
   'ja': 'プラハ',
   'kl': 'Praha',
   'ga': 'Prág',
   'mt': 'Praga',
   'th': 'ปราก',
   'fa': 'پراگ',
   'ar': 'براغ',
   'pl': 'Miasto Stołeczne Praga',
   'ru': 'Столица Прага',
   'cs': 'Hlavní město Praha',
   'ky': 'Прага',
   'ia': 'Praga',
   'ml': 'പ്രാഗ്

In [10]:
# nyní můžeme zjistit souřadnice místa, které jsme vyhledali na základě názvu

data[0]["lat"], data[0]["lon"]

(50.0874654, 14.4212535)

Pokud bychom však chtěli vyhledávat obráceně, tedy ze souřadnic získat název místa, museli bychom využít jinou funkci. Ta se nazývá "reverse geocoding" a obdobně pro ní OpenWeatherMap nabízí API.

In [ ]:
reverse_geocoding_api_url = "http://api.openweathermap.org/geo/1.0/reverse"

Její parametry jsou identické, jako API na získávání meteorologických dat.

In [1]:
import requests
import json
from decouple import config

KEY = config('USER_KEY')
reverse_geocoding_api_url = "http://api.openweathermap.org/geo/1.0/reverse"

# jako souřadnice budeme používat přibližné souřadnice středu Čech
lat = 50
lon = 14
# veškeré parametry HTTP dotazu umístíme do dictionary kde key parametru bude jeho název specifikovaný provozovatelem API

params = {
    "APPID": KEY,
    "lat": lat,
    "lon": lon
}

# nyní můžeme poslat samotnou žádost

response = requests.get(reverse_geocoding_api_url, params=params)

In [2]:
# zopakujeme stejný postup, jaku u předchozích příkladů 

print(response.status_code)
print("----------------")
print(response.content)
print("----------------")
data = json.loads(response.content)
data

200
----------------
b'[{"name":"Crostwitz - Chr\xc3\xb3s\xc4\x87icy","local_names":{"de":"Crostwitz"},"lat":51.2382996,"lon":14.2452872,"country":"DE","state":"Saxony"}]'
----------------


[{'name': 'Crostwitz - Chrósćicy',
  'local_names': {'de': 'Crostwitz'},
  'lat': 51.2382996,
  'lon': 14.2452872,
  'country': 'DE',
  'state': 'Saxony'}]

In [13]:
# nyní již můžeme zjistit, jak se jmenuje místo, jehož souřadnice jsme uvedli v HTTP dotazu na reverse geocoding API

data[0]["name"], data[0]["country"]

('Nižbor', 'CZ')

<h1>Cvičení 2</h1>

Vytvořte program, který se zeptá uživatele na místo, které bude moci zadat jako souřadnice nebo jako název místa (co zadá musí program poznat sám). Následně mu bude vypsán buď název nebo souřadnice (v závislosti na tom, co zadal) a korespondující počasí s identickými údaji, jako v předchozím cvičení. Údaje by navíc měli být popsané identifikátorem údaje a odděleny prázdným řádkem. 

In [ ]:
# místo pro Váš kód

<h1>Vzorové řešení 2</h1>

In [38]:
# spusťte pro kontrolu funkctionality

import json
import requests
from decouple import config

KEY = config('USER_KEY')

def geocode_reverse(_values, _params):
    direct_geocoding_api_url = "http://api.openweathermap.org/geo/1.0/reverse"
    params = _params.copy()
    params["lat"] = _values[0]
    params["lon"] = _values[1]
    response = requests.get(direct_geocoding_api_url, params=params)
    return json.loads(response.content) if response.status_code == 200 else print("Uknown error when sending reverse geocoding request")

def geocode_direct(_values, _params):
    reverse_geocoding_api_url = "http://api.openweathermap.org/geo/1.0/direct"
    params = _params.copy()
    params["q"] = f"{_values[0]},{_values[1]}"
    response = requests.get(reverse_geocoding_api_url, params=params)
    return json.loads(response.content) if response.status_code == 200 else print("Uknown error when sending direct geocoding request")

def parse_weather_data(lat_lon, params):
    weather_api_url = "https://api.openweathermap.org/data/2.5/weather"
    params["lat"], params["lon"] = lat_lon
    response = requests.get(weather_api_url, params=params)
    return json.loads(response.content) if response.status_code == 200 else print("Uknown error when sending weather data request")
       
def get_user_input():
    input_val = input("Enter your desired location either as lat,lon or city,country")
    input_val = input_val.split(",")
    values = []
    try:
        for val in input_val:
          values.append(float(val))  
    except ValueError:
        for val in input_val:
            values.append(str(val))
    return values 


def parse_data(_values):
    params = {"APPID": KEY}
    display_name = None
    if isinstance(_values[0],float):
        geo_data = geocode_reverse(_values, params)
        display_name = f'City: {data[0]["name"]}, Country: {data[0]["country"]}'
    else:
        geo_data = geocode_direct(_values, params)
        display_name = f'Latitude: {data[0]["lat"]}, Longtitude: { data[0]["lon"]}'
    lat_lon = [data[0]["lat"], data[0]["lon"]]
    weather_data = parse_weather_data(lat_lon, params)
    return display_name, weather_data

def extract_weather(data):
    desc = data["weather"][0]["description"]
    temp_raw = data["main"]["temp"] - 273.15
    wind_raw = data["wind"]["speed"] * 3.6
    temp = str(round(temp_raw, 2)) + " °C"
    wind = str(round(wind_raw, 2)) + " km/h"
    return [f"Description: {desc}",f"Temperature: {temp}", f"Wind speed: {wind}"]


def print_all(_list):
    for x in _list:
        print(x + "\n")


def main():
    values = get_user_input()
    display_name, weather_data = parse_data(values)
    to_print = extract_weather(weather_data)
    to_print.insert(0, display_name)  
    print_all(to_print)


if __name__ == '__main__':
    main()


Latitude: 49.999953, Longtitude: 14.0023688

Description: overcast clouds

Temperature: 22.72 °C

Wind speed: 12.82 km/h

